# Overview

This Notebook demonstrates a minimal example of querying the cockroach database using the engine, session, and models defined in the backend code.  The backend code is being read-only volume mounted to `/home/jovyan/backend`, and that directory has been added to the `PYTHONPATH` environment variable.  Imports should "just work".

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

logging.basicConfig()
logger = logging.getLogger("sqlalchemy.engine")
logger.propagate = True
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

# Show backend engine

In [5]:
import sqlalchemy as sa
import rich

from app.db import engine, db_session
from app.models import UserDAO, TodoDAO

engine

Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable)

<IPython.core.display.Javascript object>

# Query all Users

In [7]:
with db_session() as session:
    statement = sa.select(UserDAO)
    results = session.execute(statement)
    users = results.scalars().all()

users

INFO:sqlalchemy.engine.Engine:select current_schema()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:select version()
INFO:sqlalchemy.engine.Engine:[generated in 0.00162s] {}
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%(val)s)
INFO:sqlalchemy.engine.Engine:[generated in 0.00092s] {'val': 'sqlalchemy-cockroachdb 1.4.2'}
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%(val)s)
INFO:sqlalchemy.engine.Engine:[cached since 0.004108s ago] {'val': 'sqlalchemy 1.4'}
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT "user".id, "user".created_at, "user".name, "user".password 
FROM "user"
INFO:sqlalchemy.engine.Engine:[generated in 0.00094s] {}
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [8]:
rich.print(users[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f16765e6700>,
    'id': UUID('0afd983d-396e-4b88-8655-629d3078b010'),
    'created_at': datetime.datetime(2021, 11, 2, 18, 22, 24, 801318, 
tzinfo=datetime.timezone.utc),
    'password': 'pass',
    'name': 'user5'
}

<IPython.core.display.Javascript object>

In [9]:
for u in users:
    print(u.name)

user5
user1
user3
user9
user6
user8
user2
user7
user4
user10


<IPython.core.display.Javascript object>

# Query all Todos

In [10]:
with db_session() as session:
    statement = sa.select(TodoDAO)
    results = session.execute(statement)
    todos = results.scalars().all()
todos

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT todo.id, todo.created_at, todo.title, todo.content, todo.user_id, user_1.id AS id_1, user_1.created_at AS created_at_1, user_1.name, user_1.password 
FROM todo LEFT OUTER JOIN "user" AS user_1 ON user_1.id = todo.user_id
INFO:sqlalchemy.engine.Engine:[generated in 0.00137s] {}
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [11]:
rich.print(todos[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f167638cc70>,
    'id': UUID('1503fcbf-e498-4b2f-b732-a87b9b860350'),
    'content': 'So many Notes',
    'user_id': UUID('242a23e9-a0f2-4821-84ee-45917443b516'),
    'title': 'Note 3',
    'created_at': datetime.datetime(2021, 11, 2, 18, 22, 24, 801318, 
tzinfo=datetime.timezone.utc),
    'user': <app.models.UserDAO object at 0x7f167638cca0>
}

<IPython.core.display.Javascript object>